In [ ]:
__author__ = 'Wouter Depamelaere, Jeroen Van Der Donckt'

# Schedule Parser

This script can be used to parse the schedule into a file that is importable to Google Calendar.

### Imports

In [1]:
import csv
import urllib.request
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests

### Get the HTML

In [13]:
# Read the tables into a list of dataframes
url = 'https://people.cs.kuleuven.be/~btw/roosters1920/cws_semester_1.html'

htmlfile = urllib.request.urlopen(url)
html_text = htmlfile.read()

bs_obj = BeautifulSoup(html_text, 'lxml')
tables = bs_obj.findAll('table')

### Extract the timeperiod, locations and courses

In [14]:
dfs = []
for table in tables:
    df = pd.read_html(str(table))[0]
    dfs.append(df)
    #print(df.to_string())
print(len(dfs))
print(dfs[0])

48
             0   1            2  3                                           4
0  10:30—12:30  in  200A 00.225  :             Besturingssystemen: hoorcollege
1  10:30—12:30  in   200S 01.03  :                Frans in de bedrijfsomgeving
2  14:00—16:00  in   HIW1 00.14  :                    Philosophy of Technology
3  14:00—16:00  in   BOKU 03.22  :                    Modellering en simulatie
4  16:00—18:00  in   ELEC 01.57  :  Cryptography and Network Security: Lecture
5  16:30—18:30  in   C300 00.09  :              Declarative Languages: Lecture
6  17:30—19:30  in   200N 00.04  :        Natural Language Processing: Lecture


### Extract the date

In [17]:
# Get the page
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
soup.prettify()
# Filter all <i> tags, where dates are put in
date_tags = soup.find_all("i")
dates = []
for dt in date_tags:
    dates.append(dt.text)
    
#Strip other, none date fields that used <i> tag
del dates[0:2]
del dates[-1]

for idx, df in enumerate(dfs):
    df['datetime'] = dates[idx]
    
print(dfs[0])

             0   1            2  3  \
0  10:30—12:30  in  200A 00.225  :   
1  10:30—12:30  in   200S 01.03  :   
2  14:00—16:00  in   HIW1 00.14  :   
3  14:00—16:00  in   BOKU 03.22  :   
4  16:00—18:00  in   ELEC 01.57  :   
5  16:30—18:30  in   C300 00.09  :   
6  17:30—19:30  in   200N 00.04  :   

                                            4             datetime  
0             Besturingssystemen: hoorcollege  Maandag 14.10.2019:  
1                Frans in de bedrijfsomgeving  Maandag 14.10.2019:  
2                    Philosophy of Technology  Maandag 14.10.2019:  
3                    Modellering en simulatie  Maandag 14.10.2019:  
4  Cryptography and Network Security: Lecture  Maandag 14.10.2019:  
5              Declarative Languages: Lecture  Maandag 14.10.2019:  
6        Natural Language Processing: Lecture  Maandag 14.10.2019:  


### Select courses to follow

In [18]:
courses = ['Comparative Programming Languages: Lecture',
           'Comparative Programming Languages: Exercises',
           ]
print(courses)

['Comparative Programming Languages: Lecture', 'Comparative Programming Languages: Exercises']


### Clean and format data

In [19]:
df_schedule =  pd.concat(dfs)

del df_schedule[3] # Remove column containing :
del df_schedule[1] # Remove column containing in

# Rename columns
df_schedule.columns.values[3] = "Start Date"
df_schedule.columns.values[2] = "Subject"
df_schedule.columns.values[1] = "Location"
df_schedule.columns.values[0] = "time"

def strip_date(x):
    return x[-12:-1].replace('.',"/")
def time_24_to_12(x):
    d = datetime.strptime(x, "%H:%M")
    return d.strftime("%I:%M %p")

# Reformat "Dinsdag 28.10.2018" to "28/10/2018"
df_schedule['Start Date'] = df_schedule.apply(lambda row: strip_date(row['Start Date']), axis=1)
# Add End Date column
df_schedule['End Date'] = df_schedule['Start Date']
# Split "14:00-16:00" into two separate columns
df_schedule[['Start Time','End Time']] = df_schedule['time'].str.split('—',expand=True)
#df_schedule['Start Time'] = df_schedule['time'].str.split('—').str.get(0)
#df_schedule['End Time']   = df_schedule['time'].str.split('—').str.get(1)
# Convert 24h time to 12h time
df_schedule['Start Time'] = df_schedule.apply(lambda row: time_24_to_12(row['Start Time']), axis=1)
df_schedule['End Time']   = df_schedule.apply(lambda row: time_24_to_12(row['End Time']), axis=1)


del df_schedule['time']
df_schedule['All Day Event'] = False
df_schedule['Description'] = df_schedule['Location']
df_schedule['Private'] = False

# Filter courses
#df_schedule.Subject.str.startswith('Crypto')
df_schedule = df_schedule[df_schedule['Subject'].isin(courses)]

# Reorder columns to match Google .csv API 
columnsTitles = ['Subject', 'Start Date', 'Start Time', 'End Date', 'End Time','All Day Event', 'Description', 'Location', 'Private']
df_schedule =  df_schedule.reindex(columns=columnsTitles)

display(df_schedule)


,Subject,Start Date,Start Time,End Date,End Time,All Day Event,Description,Location,Private
11,Comparative Programming Languages: Lecture,17/10/2019,04:00 PM,17/10/2019,06:00 PM,False,200A 00.225,200A 00.225,False
5,Comparative Programming Languages: Lecture,18/10/2019,10:30 AM,18/10/2019,12:30 PM,False,200A 00.225,200A 00.225,False
11,Comparative Programming Languages: Lecture,24/10/2019,04:00 PM,24/10/2019,06:00 PM,False,200A 00.225,200A 00.225,False
5,Comparative Programming Languages: Lecture,25/10/2019,10:30 AM,25/10/2019,12:30 PM,False,200A 00.225,200A 00.225,False
3,Comparative Programming Languages: Exercises,28/10/2019,10:30 AM,28/10/2019,01:00 PM,False,200A 00.25,200A 00.25,False
4,Comparative Programming Languages: Exercises,29/10/2019,01:30 PM,29/10/2019,04:00 PM,False,200A 00.25,200A 00.25,False
12,Comparative Programming Languages: Lecture,07/11/2019,04:00 PM,07/11/2019,06:00 PM,False,200A 00.225,200A 00.225,False
5,Comparative Programming Languages: Exercises,12/11/2019,01:30 PM,12/11/2019,04:00 PM,False,200A 00.25,200A 00.25,False
11,Comparative Programming Languages: Lecture,14/11/2019,04:00 PM,14/11/2019,06:00 PM,False,200A 00.225,200A 00.225,False
4,Comparative Programming Languages: Exercises,15/11/2019,10:30 AM,15/11/2019,01:00 PM,False,200A 00.25,200A 00.25,False


### Export to .csv

In [22]:
df_schedule.to_csv('schedule.csv',index=False)